In [1]:
!pip install -q /kaggle/input/language-tool-python-2-7-1/language_tool_python-2.7.1-py3-none-any.whl

In [2]:
# %% Directory settings

# ====================================================
# Directory settings
# ====================================================
from pathlib import Path
import re

INPUT_DIR = Path("../input/")

In [3]:
import os
import zipfile
from zipfile import ZipFile
import shutil

# create download path
def get_language_tool_cache_path():
    # Get download path from environment or use default.
    download_path = os.environ.get(
        'LTP_PATH',
        os.path.join(os.path.expanduser("~"), ".cache", "language_tool_python")
    )
    # Make download path, if it doesn't exist.
    os.makedirs(download_path, exist_ok=True)
    return download_path

lt_path = get_language_tool_cache_path()
lt_path

'/root/.cache/language_tool_python'

In [4]:
#cant move files directly from input to cache, so we zip it to output and unzip again


def get_all_file_paths(directory):
  
    # initializing empty file paths list
    file_paths = []
  
    # crawling through directory and subdirectories
    for root, directories, files in os.walk(directory):
        for filename in files:
            # join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)
  
    # returning all file paths
    return file_paths        
  
def main():
    # path to folder which needs to be zipped
    directory = '../input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'
  
    # calling function to get all file paths in the directory
    file_paths = get_all_file_paths(directory)

    # writing files to a zipfile
    with ZipFile('./lt.zip','w') as zip:
        # writing each file one by one
        for file in file_paths:
            zip.write(file)
  
    print('All files zipped successfully!')        
    
main()


 
zip_file = "./lt.zip"
 
try:
    with zipfile.ZipFile(zip_file) as z:
        z.extractall()
        print("Extracted all")
except:
    print("Invalid file")
    
#move to cache
!mv {'./input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'} {lt_path} 
print(os.listdir('/root/.cache/language_tool_python/'))

#remove files from output

shutil.rmtree('./input')
os.remove("./lt.zip")

All files zipped successfully!
Extracted all
['LanguageTool-5.7']


In [5]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
import re
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import os
from sklearn.metrics.pairwise import cosine_similarity
import string
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import hstack
import spacy
import scipy.sparse as sp
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
import warnings
from collections import Counter
from itertools import islice

warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Helper functions

In [6]:
from nltk.tokenize import sent_tokenize
from collections import Counter
import spacy
import re
from textblob import Word
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import math
import language_tool_python

tool = language_tool_python.LanguageTool('en-US')
nlp = spacy.load("en_core_web_sm")

def sentence_length_variation(text):
    sentences = sent_tokenize(text)
    lengths = [len(s.split()) for s in sentences if len(s.split()) > 0]

    if len(lengths) < 2:
        return 0.0  

    return np.std(lengths)   # Standard deviation

# def vocabulary_diversity(text): ## same as type token ratio
#     words = [w.lower() for w in text.split() if w.isalpha()]
#     if len(words) == 0:
#         return 0
#     return len(set(words)) / len(words)

def repetition_rate(text):
    words = [w.lower() for w in text.split()]
    bigrams = [(words[i], words[i+1]) for i in range(len(words)-1)]
    if len(bigrams) == 0:
        return 0

    counts = Counter(bigrams)
    repeated = sum(1 for bg, c in counts.items() if c > 1)

    return repeated / len(bigrams)

def personal_voice_score(text):
    personal_pronouns = {"i", "me", "my", "mine", "we", "us", "our", "ours"}
    words = [w.lower() for w in text.split()]
    count = sum(1 for w in words if w in personal_pronouns)
    if len(words) == 0:
        return 0
    return count / len(words)

def emotion_variation(text):
    sentences = sent_tokenize(text)
    if len(sentences) < 2:
        return 0

    sentiments = [TextBlob(s).sentiment.polarity for s in sentences]
    diffs = [abs(sentiments[i] - sentiments[i+1]) for i in range(len(sentiments)-1)]

    return np.mean(diffs)


def specificity_score(text):
    doc = nlp(text)
    concrete_tags = {"NOUN", "PROPN", "NUM"}  
    concrete_count = sum(1 for token in doc if token.pos_ in concrete_tags)
    if len(doc) == 0:
        return 0
    return concrete_count / len(doc)


def imperfection_score(text):
    words = [w for w in re.findall(r"\b\w+\b", text)]
    if len(words) == 0:
        return 0

    misspelled = sum(1 for w in words if Word(w).correct().lower() != w.lower())
    return misspelled / len(words)

figurative_markers = [
    "like", "as if", "as though", "metaphor", "symbolic", 
    "resembles", "reminds me of", "figurative"
]

def figurative_language_score(text):
    t = text.lower()
    count = sum(t.count(m) for m in figurative_markers)
    return count

def paragraph_coherence_consistency(text):
    paragraphs = [p.strip() for p in text.split("\n") if len(p.strip()) > 0]

    if len(paragraphs) < 2:
        return 0

    vec = TfidfVectorizer().fit_transform(paragraphs)
    sims = []

    for i in range(len(paragraphs)-1):
        sim = cosine_similarity(vec[i], vec[i+1])[0][0]
        sims.append(sim)

    return np.mean(sims)


def predictability_score(text):
    words = [w.lower() for w in text.split()]
    counts = Counter(words)
    total = len(words)
    if total == 0:
        return 0

    probs = [counts[w]/total for w in words]
    surprise = [-math.log(p) for p in probs]

    return np.mean(surprise)

hedge_words = {
    "maybe", "perhaps", "sort of", "kind of", "i guess", 
    "probably", "possibly", "apparently", "roughly"
}

def hedge_uncertainty_score(text):
    t = text.lower()
    count = sum(t.count(hw) for hw in hedge_words)
    return count

transitions = [
    "however", "therefore", "meanwhile", "moreover", "furthermore",
    "in contrast", "on the other hand", "overall", "in summary"
]

def transition_variety_score(text):
    t = text.lower()
    count = sum(t.count(word) for word in transitions)
    return count


In [7]:
train_df = pd.read_csv('/kaggle/input/making-features-in-pan-clef-dataset/train.csv')
test_df = pd.read_csv('/kaggle/input/making-features-in-pan-clef-dataset/test.csv')

In [8]:
from textblob import TextBlob

print("Finding sentence_length_variation....")
train_df['sentence_length_variation'] = train_df['text'].apply(sentence_length_variation)
test_df['sentence_length_variation'] = test_df['text'].apply(sentence_length_variation)

print("Finding repetition_rate....")
train_df['repetition_rate'] = train_df['text'].apply(repetition_rate)
test_df['repetition_rate'] = test_df['text'].apply(repetition_rate)

print("Finding personal_voice_score....")
train_df['personal_voice_score'] = train_df['text'].apply(personal_voice_score)
test_df['personal_voice_score'] = test_df['text'].apply(personal_voice_score)

print("Finding emotion_variation....")
train_df['emotion_variation'] = train_df['text'].apply(emotion_variation)
test_df['emotion_variation'] = test_df['text'].apply(emotion_variation)

print("Finding specificity_score....")
train_df['specificity_score'] = train_df['text'].apply(specificity_score)
test_df['specificity_score'] = test_df['text'].apply(specificity_score)

# print("Finding imperfection_score....")
# train_df['imperfection_score'] = train_df['text'].apply(imperfection_score)
# test_df['imperfection_score'] = test_df['text'].apply(imperfection_score)

print("Finding figurative_language_score....")
train_df['figurative_language_score'] = train_df['text'].apply(figurative_language_score)
test_df['figurative_language_score'] = test_df['text'].apply(figurative_language_score)

print("Finding paragraph_coherence_consistency....")
train_df['paragraph_coherence_consistency'] = train_df['text'].apply(paragraph_coherence_consistency)
test_df['paragraph_coherence_consistency'] = test_df['text'].apply(paragraph_coherence_consistency)

print("Finding predictability_score....")
train_df['predictability_score'] = train_df['text'].apply(predictability_score)
test_df['predictability_score'] = test_df['text'].apply(predictability_score)

print("Finding hedge_uncertainty_score....")
train_df['hedge_uncertainty_score'] = train_df['text'].apply(hedge_uncertainty_score)
test_df['hedge_uncertainty_score'] = test_df['text'].apply(hedge_uncertainty_score)

print("Finding transition_variety_score....")
train_df['transition_variety_score'] = train_df['text'].apply(transition_variety_score)
test_df['transition_variety_score'] = test_df['text'].apply(transition_variety_score)

Finding sentence_length_variation....
Finding repetition_rate....
Finding personal_voice_score....
Finding emotion_variation....
Finding specificity_score....
Finding figurative_language_score....
Finding paragraph_coherence_consistency....
Finding predictability_score....
Finding hedge_uncertainty_score....
Finding transition_variety_score....


In [9]:
def grammatical_mistakes(sentence):

  mistakes = len(tool.check(sentence))

  return mistakes

In [10]:
train_df['grammatical_mistakes'] = train_df['text'].apply(grammatical_mistakes)
test_df['grammatical_mistakes'] = test_df['text'].apply(grammatical_mistakes)

In [11]:
def pos_tag_ngrams(text, n=2):
    doc = nlp(text)
    tags = [token.pos_ for token in doc if token.is_alpha]

    if len(tags) < n:
        return {}

    ngrams = zip(*[tags[i:] for i in range(n)])
    return Counter(ngrams)

def pos_ngram_variety(text, n=2):
    ngrams = pos_tag_ngrams(text, n)
    return len(ngrams)

In [12]:
train_df['pos_2gram_variety'] = train_df['text'].apply(pos_ngram_variety)
test_df['pos_2gram_variety'] = test_df['text'].apply(pos_ngram_variety)
## number of 3 grams
train_df['pos_3gram_variety'] = train_df['text'].apply(lambda x: pos_ngram_variety(x, n=3))
test_df['pos_3gram_variety'] = test_df['text'].apply(lambda x: pos_ngram_variety(x, n=3))
## number of 4 grams
train_df['pos_4gram_variety'] = train_df['text'].apply(lambda x: pos_ngram_variety(x, n=4))
test_df['pos_4gram_variety'] = test_df['text'].apply(lambda x: pos_ngram_variety(x, n=4))

In [13]:
# import torch
# from transformers import GPT2LMHeadModel, GPT2TokenizerFast

# # Load once (place globally)
# tokenizer = GPT2TokenizerFast.from_pretrained("/kaggle/input/gpt2/transformers/gpt21/1")
# gpt_model = GPT2LMHeadModel.from_pretrained("/kaggle/input/gpt2/transformers/gpt21/1")
# gpt_model.eval()

# def perplexity(text):
#     enc = tokenizer(text, return_tensors="pt", truncation=True, max_length=1024)
#     input_ids = enc.input_ids

#     with torch.no_grad():
#         outputs = gpt_model(input_ids, labels=input_ids)

#     loss = outputs.loss
#     return torch.exp(loss).item()

In [14]:
# train_df['perplexity'] = train_df['text'].apply(perplexity)
# test_df['perplexity'] = test_df['text'].apply(perplexity)

In [15]:
train_df.to_csv('train.csv',index=False)
test_df.to_csv('test.csv',index=False)